In [ ]:
!ls
!pwd
#/kaggle/working

import os
os.environ['WANDB_MODE'] = 'disabled'

**Setting Up the Environment**

In [2]:
!hf auth login --token #

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `hf`CLI if you want to set the git credential as well.
Token is valid (permission: fineGrained).
The token `Token1` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `Token1`


In [3]:
!git clone https://github.com/drn-i/WESAD
!mv WESAD/* .

Cloning into 'WESAD'...
remote: Enumerating objects: 84, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 84 (delta 9), reused 0 (delta 0), pack-reused 61 (from 1)
Receiving objects: 100% (84/84), 1008.20 KiB | 36.01 MiB/s, done.
Resolving deltas: 100% (35/35), done.


In [4]:
!pip install transformers>=4.44.0 datasets>=2.20.0 peft>=0.11.0 accelerate>=0.33.0 bitsandbytes>=0.43.0 pandas>=2.2.0 scikit-learn>=1.4.0 numpy>=1.26.0 joblib>=1.3.0 tqdm>=4.66.0
!pip install scipy matplotlib seaborn cvxopt xgboost shap kaggle
#!pip install -r /requirements.txt

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
pylibcudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 21.0.0 which is incompatible.
cudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 21.0.0 which is incompatible.
bigframes 2.12.0 requires google-cloud-bigquery[bqstorage,pandas]>=3.31.0, but you have google-cloud-bigquery 3.25.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.1.0 which is incompatible.
libcugraph-cu12 25.6.0 requires libraft-cu12==25.6.*, but you have libraft-cu12 25.2.0 which is incompatible.
gradio 5.38.1 requires pydantic<2.12,>=2.0, but you have pydantic 2.12.0a1 which is incompatible.
cudf-polars-cu12

**Data Preparation**

In [5]:
# Prep 1: WESAD with Top-15 Features
!python prepare_data.py --mode wesad-top

# Prep 2: WESAD with Full Features
!python prepare_data.py --mode wesad-full

--- Processing WESAD (mode: top) ---
Using Top 15 features.
Performing 80/20 train-test split (stratified)...
Successfully saved 786 samples to train_dataset_top.jsonl
Successfully saved 197 samples to wesad_test_dataset_top.jsonl
WESAD preparation complete.
--- Processing WESAD (mode: full) ---
Using 56 features.
Performing 80/20 train-test split (stratified)...
Successfully saved 786 samples to train_dataset_full.jsonl
Successfully saved 197 samples to wesad_test_dataset_full.jsonl
WESAD preparation complete.


In [6]:
# Prep 3: Hypertension Test Set
!python prepare_data.py --mode hypertension

--- Processing Hypertension (mode: test) ---
Successfully saved 1985 samples to hypertension_test_dataset.jsonl
Hypertension preparation complete.


**Training & Evaluation on Top Features** (1B Model)

In [15]:
# 1. Train the model on 'top' features
!python train.py \
  --model_name "meta-llama/Llama-3.2-1B-Instruct" \
  --dataset_file "train_dataset_top.jsonl" \
  --output_dir "/kaggle/temp/model-1b-top-features"

2025-10-21 16:30:12.738374: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761064212.770429     251 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761064212.777567     251 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
--- Starting Training ---
Model: meta-llama/Llama-3.2-1B-Instruct
Dataset: train_dataset_top.jsonl
Output: /kaggle/temp/model-1b-top-features
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be

In [16]:
# 2. Evaluate on the WESAD (top) test set (Step 1)
!python evaluate.py \
  --model_path "/kaggle/temp/model-1b-top-features" \
  --dataset_file "wesad_test_dataset_top.jsonl"

--- Starting Evaluation ---
Model: /kaggle/temp/model-1b-top-features
Test File: wesad_test_dataset_top.jsonl
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `An

In [17]:
# 3. Evaluate on the Hypertension test set (Step 2)
!python evaluate.py \
  --model_path "/kaggle/temp/model-1b-top-features" \
  --dataset_file "hypertension_test_dataset.jsonl"

--- Starting Evaluation ---
Model: /kaggle/temp/model-1b-top-features
Test File: hypertension_test_dataset.jsonl
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using 

**Training & Evaluation on All Features**  (1B Model)

In [20]:
# 1. Train the model on 'full' features
!python train.py \
  --model_name "meta-llama/Llama-3.2-1B-Instruct" \
  --dataset_file "train_dataset_full.jsonl" \
  --output_dir "/kaggle/temp/model-1b-full-features"

2025-10-21 17:21:12.897608: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761067272.920750     355 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761067272.927572     355 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
--- Starting Training ---
Model: meta-llama/Llama-3.2-1B-Instruct
Dataset: train_dataset_full.jsonl
Output: /kaggle/temp/model-1b-full-features
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only 

In [21]:
# 2. Evaluate on the WESAD (full) test set (Step 1)
!python evaluate.py \
  --model_path "/kaggle/temp/model-1b-full-features" \
  --dataset_file "wesad_test_dataset_full.jsonl"

--- Starting Evaluation ---
Model: /kaggle/temp/model-1b-full-features
Test File: wesad_test_dataset_full.jsonl
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `

In [23]:
# 3. Evaluate on the Hypertension test set (Step 2)
!python evaluate.py \
  --model_path "/kaggle/temp/model-1b-full-features" \
  --dataset_file "hypertension_test_dataset.jsonl"

--- Starting Evaluation ---
Model: /kaggle/temp/model-1b-full-features
Test File: hypertension_test_dataset.jsonl
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using

**Training & Evaluation on Top Features** (8B Model)

In [7]:
# 1. Train the model on 'top' features
!python train.py \
  --model_name "meta-llama/Llama-3.1-8B-Instruct" \
  --dataset_file "train_dataset_top.jsonl" \
  --output_dir "/kaggle/temp/model-8b-top-features"

2025-10-21 20:06:32.837976: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761077193.050678     147 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761077193.111835     147 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
--- Starting Training ---
Model: meta-llama/Llama-3.1-8B-Instruct
Dataset: train_dataset_top.jsonl
Output: /kaggle/temp/model-8b-top-features
tokenizer_config.json: 100%|████████████████| 55.4k/55.4k [00:00<00:00, 110MB/s]
tokenizer.json: 100%|██████████████████████| 9.09M/9.09M [00:02<00:00, 3.53MB/s]
config.json: 100%|█████████████████████████████| 855/855 [00:00<00:00, 4.17MB/s]
/usr/local/lib/python3.11/dist-packages/pydantic/_in

In [8]:
# 2. Evaluate on the WESAD (top) test set (Step 1)
!python evaluate.py \
  --model_path "/kaggle/temp/model-8b-top-features" \
  --dataset_file "wesad_test_dataset_top.jsonl"

--- Starting Evaluation ---
Model: /kaggle/temp/model-8b-top-features
Test File: wesad_test_dataset_top.jsonl
An error occurred loading the model: Repo id must be in the form 'repo_name' or 'namespace/repo_name': '/kaggle/temp/model-8b-top-features'. Use `repo_type` argument if needed.


In [9]:
# 3. Evaluate on the Hypertension test set (Step 2)
!python evaluate.py \
  --model_path "/kaggle/temp/model-8b-top-features" \
  --dataset_file "hypertension_test_dataset.jsonl"

--- Starting Evaluation ---
Model: /kaggle/temp/model-8b-top-features
Test File: hypertension_test_dataset.jsonl
An error occurred loading the model: Repo id must be in the form 'repo_name' or 'namespace/repo_name': '/kaggle/temp/model-8b-top-features'. Use `repo_type` argument if needed.


**Training & Evaluation on All Features**  (8B Model)

In [10]:
# 1. Train the model on 'full' features
!python train.py \
  --model_name "meta-llama/Llama-3.1-8B-Instruct" \
  --dataset_file "train_dataset_full.jsonl" \
  --output_dir "/kaggle/temp/model-8b-full-features"

2025-10-21 20:10:02.918696: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761077402.945402     234 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761077402.953193     234 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
--- Starting Training ---
Model: meta-llama/Llama-3.1-8B-Instruct
Dataset: train_dataset_full.jsonl
Output: /kaggle/temp/model-8b-full-features
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only 

In [11]:
# 2. Evaluate on the WESAD (full) test set (Step 1)
!python evaluate.py \
  --model_path "/kaggle/temp/model-8b-full-features" \
  --dataset_file "wesad_test_dataset_full.jsonl"

--- Starting Evaluation ---
Model: /kaggle/temp/model-8b-full-features
Test File: wesad_test_dataset_full.jsonl
An error occurred loading the model: Repo id must be in the form 'repo_name' or 'namespace/repo_name': '/kaggle/temp/model-8b-full-features'. Use `repo_type` argument if needed.


In [12]:
# 3. Evaluate on the Hypertension test set (Step 2)
!python evaluate.py \
  --model_path "/kaggle/temp/model-8b-full-features" \
  --dataset_file "hypertension_test_dataset.jsonl"

--- Starting Evaluation ---
Model: /kaggle/temp/model-8b-full-features
Test File: hypertension_test_dataset.jsonl
An error occurred loading the model: Repo id must be in the form 'repo_name' or 'namespace/repo_name': '/kaggle/temp/model-8b-full-features'. Use `repo_type` argument if needed.
